# SD-JWT for Certification of Registration credential in Domain validation usecase with ODI

This is the SD-JWT payload creation playground for the domain validation usecase specified at:   https://hackmd.io/@lalchandran/ryB54F_Pn

## Setup cryptographic keys

In [ ]:
import dataclasses
import asyncio
import json
from rich.console import Console
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK

console = Console()
loop = asyncio.get_event_loop()

# Generate did:key identifier and associated public and privatekey pairs from the cryptographic seed.
cryptographic_seed = b'helloworld'
issuer_did = KeyDid(seed=cryptographic_seed)
issuer_did.create_keypair()

console.log('Issuer public key JWK:', issuer_did.public_key_jwk)

[22:21:16] Issuer public key JWK:                                                                  ]8;id=198899;file:///tmp/ipykernel_38447/3472563506.py\3472563506.py]8;;\:]8;id=97172;file:///tmp/ipykernel_38447/3472563506.py#15\15]8;;\
           {                                                                                                       
               'kty': 'EC',                                                                                        
               'kid': 'tJ7_O_Xu862O_9L4oxdBSkRotGJ2Id9smSovHtnUUo8',                                               
               'crv': 'P-256',                                                                                     
               'x': 'PTKECXSeSzIRRjYBKPECaRBRp1VH_g2EztKLJf8jpVc',                                                 
               'y': 'HHpPqdZ8P0ZVz9CSryp7FvmtTTnNImmlQuiatJncoNM'                                                  
           }                                                                                                       

## Claims to be issued

In [ ]:
certificate_of_registration_claims = {
  "legalStatus": "ACTIVE",
  "legalForm": "Aktiebolag",
  "activity": "Real Estate",
  "orgNumber": "559133-2720",
  "registrationDate": "2005-10-08",
  "registeredAddress": {
    "fullAddress": "123 Main St",
    "thoroughFare": "Sveavägen",
    "postName": "Stockholm",
    "locatorDesignator": "111 34",
    "code": "48",
    "adminUnitLevel1": "SE"
  },
  "attestationValidity": "31-Dec-2022"
}

console.log("Certificate of Registration claims to be issued: ", certificate_of_registration_claims)

           Certificate of Registration claims to be issued:                                        ]8;id=370641;file:///tmp/ipykernel_38447/1992885474.py\1992885474.py]8;;\:]8;id=386988;file:///tmp/ipykernel_38447/1992885474.py#18\18]8;;\
           {                                                                                                       
               'legalStatus': 'ACTIVE',                                                                            
               'legalForm': 'Aktiebolag',                                                                          
               'activity': 'Real Estate',                                                                          
               'orgNumber': '559133-2720',                                                                         
               'registrationDate': '2005-10-08',                                                                   
               'registeredAddress': {                                                                              
                   'fullAddress': '123 Main St',                                                                   
                   'thoroughFare': 'Sveavägen',                                                                    
                   'postName': 'Stockholm',                                                                        
                   'locatorDesignator': '111 34',                                                                  
                   'code': '48',                                                                                   
                   'adminUnitLevel1': 'SE'                                                                         
               },                                                                                                  
               'attestationValidity': '31-Dec-2022'                                                                
           }                                                                                                       

## Calculate full disclosure for claims to be issued

In [ ]:
import json
import base64

disclosure = base64.urlsafe_b64encode(json.dumps(certificate_of_registration_claims).encode('utf-8')).decode('utf-8').rstrip("=")
console.log("Disclosure: ", disclosure)

           Disclosure:                                                                              ]8;id=565018;file:///tmp/ipykernel_38447/2974318599.py\2974318599.py]8;;\:]8;id=180152;file:///tmp/ipykernel_38447/2974318599.py#5\5]8;;\
           eyJsZWdhbFN0YXR1cyI6ICJBQ1RJVkUiLCAibGVnYWxGb3JtIjogIkFrdGllYm9sYWciLCAiYWN0aXZpdHkiOiAi                
           UmVhbCBFc3RhdGUiLCAib3JnTnVtYmVyIjogIjU1OTEzMy0yNzIwIiwgInJlZ2lzdHJhdGlvbkRhdGUiOiAiMjAw                
           NS0xMC0wOCIsICJyZWdpc3RlcmVkQWRkcmVzcyI6IHsiZnVsbEFkZHJlc3MiOiAiMTIzIE1haW4gU3QiLCAidGhv                
           cm91Z2hGYXJlIjogIlN2ZWF2XHUwMGU0Z2VuIiwgInBvc3ROYW1lIjogIlN0b2NraG9sbSIsICJsb2NhdG9yRGVz                
           aWduYXRvciI6ICIxMTEgMzQiLCAiY29kZSI6ICI0OCIsICJhZG1pblVuaXRMZXZlbDEiOiAiU0UifSwgImF0dGVz                
           dGF0aW9uVmFsaWRpdHkiOiAiMzEtRGVjLTIwMjIifQ                                                              

## Calculate disclosure SHA-256 digest

In [ ]:
import hashlib
import base64

sha256_hash = hashlib.sha256()
sha256_hash.update(disclosure.encode('utf-8'))
sha256_digest = sha256_hash.digest()

disclosure_sha_256 = base64.urlsafe_b64encode(sha256_digest).decode('utf-8').rstrip("=")
console.log("Disclosure SHA-256: ", disclosure_sha_256)

           Disclosure SHA-256:  JhDIF1oJsAhyPf4p9p97h6iSEH5Nfbj7eq_L8tXHfsY                         ]8;id=459793;file:///tmp/ipykernel_38447/2709858298.py\2709858298.py]8;;\:]8;id=381739;file:///tmp/ipykernel_38447/2709858298.py#9\9]8;;\

## Create SD-JWT

In [ ]:
import json
from jwcrypto import jwt

_sd = [disclosure_sha_256]
sd_jwt = issuer_did.generate_sd_jwt(_sd)

console.log("SD-JWT: ", sd_jwt)

sd_jwt_deserialised = jwt.JWT(key=issuer_did._key, jwt=sd_jwt)
console.log("SD-JWT payload: ", json.loads(sd_jwt_deserialised.claims))
console.log("SD-JWT header: ",  json.loads(sd_jwt_deserialised.header))

           SD-JWT:                                                                                  ]8;id=378171;file:///tmp/ipykernel_38447/1335579605.py\1335579605.py]8;;\:]8;id=396022;file:///tmp/ipykernel_38447/1335579605.py#7\7]8;;\
           eyJhbGciOiJFUzI1NiJ9.eyJfc2QiOlsiSmhESUYxb0pzQWh5UGY0cDlwOTdoNmlTRUg1TmZiajdlcV9MOHRYSGZ                
           zWSJdLCJfc2RfYWxnIjoic2hhLTI1NiIsImV4cCI6MTY4NzI4MzQ3NiwiaWF0IjoxNjg3Mjc5ODc2LCJpc3MiOiJ                
           odHRwczovL2lzc3Vlci5pZ3JhbnQuaW8ifQ.em_pTukG4TB96-L3BsCtfTYjcNq7vjXDS6v6fjkcFu6DIvCNXvDR                
           k4CDiK9wwc7TlyN6cSEx7axWMdhAD-_cjw                                                                      

           SD-JWT payload:                                                                         ]8;id=684716;file:///tmp/ipykernel_38447/1335579605.py\1335579605.py]8;;\:]8;id=820006;file:///tmp/ipykernel_38447/1335579605.py#10\10]8;;\
           {                                                                                                       
               '_sd': ['JhDIF1oJsAhyPf4p9p97h6iSEH5Nfbj7eq_L8tXHfsY'],                                             
               '_sd_alg': 'sha-256',                                                                               
               'exp': 1687283476,                                                                                  
               'iat': 1687279876,                                                                                  
               'iss': 'https://issuer.igrant.io'                                                                   
           }                                                                                                       

           SD-JWT header:                                                                          ]8;id=591760;file:///tmp/ipykernel_38447/1335579605.py\1335579605.py]8;;\:]8;id=872588;file:///tmp/ipykernel_38447/1335579605.py#11\11]8;;\
           {'alg': 'ES256'}                                                                                        

## Create combined format for issuance (a.k.a issue credential to holder)

*Note: Format is `<SD-JWT>~<Disclosure 1>~<Disclosure 2>~...~<Disclosure N>`*

In [ ]:
combined_format_for_issuance = sd_jwt + "~" + disclosure_sha_256
console.log("Combined format for issuance : ", combined_format_for_issuance)

           Combined format for issuance :                                                           ]8;id=776602;file:///tmp/ipykernel_38447/2596184796.py\2596184796.py]8;;\:]8;id=572349;file:///tmp/ipykernel_38447/2596184796.py#2\2]8;;\
           eyJhbGciOiJFUzI1NiJ9.eyJfc2QiOlsiSmhESUYxb0pzQWh5UGY0cDlwOTdoNmlTRUg1TmZiajdlcV9MOHRYSGZ                
           zWSJdLCJfc2RfYWxnIjoic2hhLTI1NiIsImV4cCI6MTY4NzI4MzQ3NiwiaWF0IjoxNjg3Mjc5ODc2LCJpc3MiOiJ                
           odHRwczovL2lzc3Vlci5pZ3JhbnQuaW8ifQ.em_pTukG4TB96-L3BsCtfTYjcNq7vjXDS6v6fjkcFu6DIvCNXvDR                
           k4CDiK9wwc7TlyN6cSEx7axWMdhAD-_cjw~JhDIF1oJsAhyPf4p9p97h6iSEH5Nfbj7eq_L8tXHfsY                          